In [83]:
tfIdfNum = 64#100
featureNum = tfIdfNum*2+2-1
base_path = '/Users/yungchuanlee/Documents/learn/kaggle/QuoraQuestion'
trainCsvPath = base_path + '/trainTfidf_%s.csv'
testCsvPath = base_path  + '/testTfidf_%s_%s.csv'

In [84]:
#read train csv
import numpy as np
np.random.seed(1337)

''' Read train files '''
train_data = np.genfromtxt(trainCsvPath % tfIdfNum, delimiter=',',skip_header=0)
#train: 128 + 2 + label
''' The first column to the 130th(ignore weight 1k) column is used as input features '''
X_train = train_data[:,0:featureNum]
X_train = X_train.astype('float32')

''' The 131-th column is the answer '''
y_train = train_data[:,len(train_data[0,:])-1]
y_train = y_train.astype('int')

''' Shuffle training data '''
from sklearn.utils import shuffle
X_train,Y_train = shuffle(X_train,y_train,random_state=100)

In [85]:
y_train

array([0, 1, 1, ..., 0, 0, 1])

In [86]:
len(train_data[0,:])
# Y_train.shape

131

In [87]:
#deep learning - declare model

''' set the size of mini-batch and number of epochs'''
batch_size = 16
epochs = 30

''' Import keras to build a DL model '''
from keras.models import Sequential
from keras.layers.core import Dense, Activation
''' Import l1,l2 (regularizer) '''
from keras.regularizers import l1,l2
''' EarlyStopping '''
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 3)

print('Building a model whose optimizer="Adam", activation function=relu')
model_adam = Sequential()
# model_adam.add(Dense(128, input_dim=featureNum, kernel_regularizer=l2(0.005) ))
model_adam.add(Dense(128, input_dim=featureNum ))
model_adam.add(Activation('relu'))
# model_adam.add(Dense(256, kernel_regularizer=l2(0.005)))
model_adam.add(Dense(256))
model_adam.add(Activation('relu'))
# model_adam.add(Dense(1, kernel_regularizer=l2(0.005)))
model_adam.add(Dense(1))
model_adam.add(Activation('sigmoid'))

Building a model whose optimizer="Adam", activation function=relu


In [88]:
# model fitting
''' Setting optimizer as Adam '''
from keras.optimizers import SGD, Adam, RMSprop, Adagrad
model_adam.compile(loss= 'binary_crossentropy',optimizer='Adam', metrics=['accuracy'])

'''Fit models and use validation_split=0.1 '''
history_adam = model_adam.fit(X_train, Y_train, batch_size=batch_size,
                              epochs=epochs,verbose=0,shuffle=True,validation_split=0.1)
                             #,callbacks=[early_stopping])

loss_adam= history_adam.history.get('loss')
acc_adam = history_adam.history.get('acc')

In [89]:
#plot
''' Visualize the loss and accuracy of both models'''
import matplotlib.pyplot as plt
plt.figure(0)
plt.subplot(121)
plt.plot(range(len(loss_adam)), loss_adam,label='Adam')
plt.title('Loss')
plt.legend(loc='upper left')
plt.subplot(122)
plt.plot(range(len(acc_adam)), acc_adam,label='Adam')
plt.title('Accuracy')
plt.savefig(base_path  + '/04_optimizerSelection_%s.png' % tfIdfNum ,dpi=300,format='png')
plt.close()
print('Result saved into 04_optimizerSelection.png')

Result saved into 04_optimizerSelection.png


In [90]:
#save model
from keras.models import model_from_json
model_json = model_adam.to_json()
with open(base_path  + "model_%s.json" % tfIdfNum, "w") as json_file:
    json_file.write(model_json)


In [91]:
# #load model
# json_file = open(base_path  + "model_%s.json" % tfIdfNum, 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model_adam = model_from_json(loaded_model_json)

In [92]:
#read test csv & predict
import numpy as np
import pandas as pd
np.random.seed(1337)

pdArr = []

for fragNum in range(0,10):
#for fragNum in range(1,6):
    test_data = np.genfromtxt(testCsvPath % (tfIdfNum, fragNum), delimiter=',',skip_header=0)
    X_test = test_data[:,1:featureNum+1]
    X_test = X_test.astype('float32')
    preds = model_adam.predict(X_test)
    test_labels = test_data[:,0]
    test_labels = test_labels.astype('int')

    out_df = pd.DataFrame({"test_id":test_labels,"is_duplicate":preds.ravel()})
    pdArr.append(out_df)

In [93]:
res_pd = pd.concat(pdArr)

In [94]:
res_pd[['test_id','is_duplicate']].to_csv(base_path + '/DL_predictions_%s.csv' % tfIdfNum, index=False)

In [95]:
# #read test csv
# import numpy as np
# import pandas as pd
# np.random.seed(1337)

# fragNum = 1
# ''' Read test files '''
# test_data = np.genfromtxt(testCsvPath % (tfIdfNum, fragNum), delimiter=',',skip_header=0)
# X_test = test_data[:,1:featureNum+1]
# X_test_1 = X_test.astype('float32')
# preds = model_adam.predict(X_test_1)
# print(preds.shape)

# test_labels_1 = test_data[:,0]
# test_labels_1 = test_labels_1.astype('int')

# out_df_1 = pd.DataFrame({"test_id":test_labels_1,"is_duplicate":preds.ravel()})

In [96]:
X_test.shape

(231520, 129)

In [97]:
# fragNum = 2
# ''' Read test files '''
# test_data = np.genfromtxt(testCsvPath % (tfIdfNum, fragNum), delimiter=',',skip_header=0)
# X_test = test_data[:,1:featureNum+1]
# X_test_2 = X_test.astype('float32')
# preds = model_adam.predict(X_test_2)
# print(preds.shape)

# test_labels_2 = test_data[:,0]
# test_labels_2 = test_labels_2.astype('int')

# out_df_2 = pd.DataFrame({"test_id":test_labels_2,"is_duplicate":preds.ravel()})

In [98]:
# fragNum = 3
# ''' Read test files '''
# test_data = np.genfromtxt(testCsvPath % (tfIdfNum, fragNum), delimiter=',',skip_header=0)
# X_test = test_data[:,1:featureNum+1]
# X_test_3 = X_test.astype('float32')
# preds = model_adam.predict(X_test_3)
# print(preds.shape)

# test_labels_3 = test_data[:,0]
# test_labels_3 = test_labels_3.astype('int')

# out_df_3 = pd.DataFrame({"test_id":test_labels_3,"is_duplicate":preds.ravel()})

In [99]:
# fragNum = 4
# ''' Read test files '''
# test_data = np.genfromtxt(testCsvPath % (tfIdfNum, fragNum), delimiter=',',skip_header=0)
# X_test = test_data[:,1:featureNum+1]
# X_test_4 = X_test.astype('float32')
# preds = model_adam.predict(X_test_4)
# print(preds.shape)

# test_labels_4 = test_data[:,0]
# test_labels_4 = test_labels_4.astype('int')

# out_df_4 = pd.DataFrame({"test_id":test_labels_4,"is_duplicate":preds.ravel()})

In [100]:
# fragNum = 5
# ''' Read test files '''
# test_data = np.genfromtxt(testCsvPath % (tfIdfNum, fragNum), delimiter=',',skip_header=0)
# X_test = test_data[:,1:featureNum+1]
# X_test_5 = X_test.astype('float32')
# preds = model_adam.predict(X_test_5)
# print(preds.shape)

# test_labels_5 = test_data[:,0]
# test_labels_5 = test_labels_5.astype('int')

# out_df_5 = pd.DataFrame({"test_id":test_labels_5,"is_duplicate":preds.ravel()})

In [101]:
# res_pd = pd.concat([out_df_1, out_df_2, out_df_3, out_df_4, out_df_5])

In [102]:
# res_pd[['test_id','is_duplicate']].to_csv(base_path + '/DL_predictions_%s.csv' % tfIdfNum, index=False)

In [103]:
 res_pd

,is_duplicate,test_id
0,1.432350e-01,0
1,3.675404e-01,1
2,9.247481e-01,2
3,4.183708e-01,3
4,2.857536e-01,4
5,6.877857e-06,5
6,6.637211e-01,6
7,9.999967e-01,7
8,3.748116e-01,8
9,2.685608e-03,9
